In [1]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("2.1.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=60d562658aa33d076321de2a57db91c7e0525a5328a75228ed9502dba7e1a8a3
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=a6a18d539c581f8983152406ada9cb9ad6c7a0b08be5faa79058e53bb7b0b41a
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt210/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 60.2 MB/s eta 0:00:00


In [2]:
!pip install matplotlib==3.5.2
!pip install git+https://github.com/openai/CLIP.git
#!pip install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1

!pip install kaolin==0.15.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.1.0_cu121.html
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 36.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
plotnine 0.12.4 requires matplotlib>=3.6.0, but you have matplotlib 3.5.2 which is incompatible.


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-p1q_5jz6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-p1q_5jz6
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=ed57f3495196f801a3142c5dd58ed687155066c497eff4ced6d98c00a81b777e
  Stored in directory: /tmp/pip-ephem-wheel-cache-zr0tiod1/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.1.0_cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.7/255.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:

import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from math import radians
from pytorch3d.transforms import axis_angle_to_matrix
# Util function for loading point clouds|
import numpy as np

# Data structures and functions for rendering
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.renderer import (
    look_at_rotation,
    look_at_view_transform,
    FoVOrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor,
    FoVPerspectiveCameras,
    PerspectiveCameras,
    rotate_on_spot,
    OrthographicCameras
)

In [12]:
def render_point_cloud(point_cloud,distance,elev,azim,a1,a2,a3):

  R, T = look_at_view_transform(distance, elev, azim)

  angles = [radians(a1), radians(a2),radians(a3)]
  rotation = axis_angle_to_matrix(torch.FloatTensor(angles))
  R, T = rotate_on_spot(R, T, rotation)
  cameras = FoVPerspectiveCameras(device=device, R=R, T=T, znear=0.01)#questa la fa di fronte

  # Define the settings for rasterization and shading. Here we set the output image to be of size
  # 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
  # and blur_radius=0.0. Refer to raster_points.py for explanations of these parameters.
  raster_settings = PointsRasterizationSettings(
      image_size=512,
      radius = 0.003,
      points_per_pixel = 10
  )
  renderer = PointsRenderer(
      rasterizer=PointsRasterizer(cameras=cameras, raster_settings=raster_settings),
      compositor=NormWeightedCompositor()
  )

  return renderer(point_cloud)


def resize(image, res):
  resized=image.permute(3, 1, 2, 0).squeeze(3)
  transform = transforms.Resize((res, res))
  out=transform(resized)
  return out


def generate_static_views(point_cloud, res):
  images = [ resize(render_point_cloud(point_cloud,3,0,0,0,0,0),res),
             resize(render_point_cloud(point_cloud,3,0,0,0,0,540),res),
             resize(render_point_cloud(point_cloud,3,90,0,0,0,0),res),
             resize(render_point_cloud(point_cloud,3,-80,-2,0,0,0),res),
             resize(render_point_cloud(point_cloud,3,0,50,0,0,0),res)
  ]

  images = torch.stack(images)
  return images

def generate_random_view(point_cloud, res, n_views, std=1):
  a1 = torch.randn(n_views) *90
  a2 = torch.randn(n_views) * 90
  a3 = torch.randn(n_views) * 90
  images = []
  for i in range(n_views):
      images.append(resize(render_point_cloud(point_cloud,3,0,0,a1[i],a2[i],a3[i]),res))

  images = torch.stack(images)
  return images



In [5]:
# Setup

import open3d as o3d
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


def segment2rgb(pred_class, colors):
    pred_rgb = torch.zeros(pred_class.shape[0], 3).to(device)
    for class_idx, color in enumerate(colors):
        pred_rgb += torch.matmul(pred_class[:,class_idx].unsqueeze(1), color.unsqueeze(0))

    return pred_rgb

def color_point_cloud(pred_class, verts, colors):
    pred_rgb = segment2rgb(pred_class, colors)
    #print(pred_rgb.shape)
    #print(verts.shape[0])

    return Pointclouds(points=[verts], features=[pred_rgb])




In [6]:
#!pip install --upgrade pip
!git clone https://github.com/rasenqt/AML_Project_2024.git
#!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2


Cloning into 'AML_Project_2024'...
remote: Enumerating objects: 976, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 976 (delta 79), reused 198 (delta 75), pack-reused 774
Receiving objects: 100% (976/976), 203.42 MiB | 20.46 MiB/s, done.
Resolving deltas: 100% (246/246), done.
Updating files: 100% (835/835), done.


# OPTIMIZE POINT CLOUD


In [7]:
import argparse
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
import sys
import os
import argparse
sys.path.append('/content/AML_Project_2024/3DHighlighter-main')
from itertools import permutations, product
from neural_highlighter import NeuralHighlighter
from Normalization import MeshNormalizer,PointCloudNormalizer
from mesh import Mesh
from pathlib import Path
from render import Renderer
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device, color_mesh
import open3d as o3d



def optimize_point_cloud(args):
    # Constrain most sources of randomness
    # (some torch backwards functions within CLIP are non-determinstic)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    # Load CLIP model
    clip_model, preprocess = clip.load(args.clipmodel, device, jit=args.jit)

    # Adjust output resolution depending on model type
    res = 224
    if args.clipmodel == "ViT-L/14@336px":
        res = 336
    if args.clipmodel == "RN50x4":
        res = 288
    if args.clipmodel == "RN50x16":
        res = 384
    if args.clipmodel == "RN50x64":
        res = 448

    Path(os.path.join(args.output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

    objbase, extension = os.path.splitext(os.path.basename(args.obj_path))
    pointcloud = o3d.io.read_point_cloud(args.obj_path)
    verts = torch.Tensor(np.asarray(pointcloud.points)).to(device)

    print("Verts shape")
    print(verts.shape)


    # Manually set RGB color (assuming you want to set all points to red)
    manual_color = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32, device=device)

    # Repeat the manual color for each vertex
    rgb_colors = manual_color.repeat(verts.shape[0], 1)

    shift = torch.mean(verts, dim=0)
    scale = torch.max(torch.norm(verts-shift, p=2, dim=1))
    verts=(verts-shift)/scale

    point_cloud = Pointclouds(points=[verts], features=[rgb_colors])
    #PointCloudNormalizer(point_cloud)()




    # Initialize variables
    background = None
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    n_augs = args.n_augs
    dir = args.output_dir

    # Record command line arguments
    with open(os.path.join(dir, 'commandline_args.txt'), 'w') as f:
        json.dump(args.__dict__, f, indent=2)

    # CLIP and Augmentation Transforms
    clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    clip_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((res, res)),
        clip_normalizer
    ])
    augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
    ])

    # MLP Settings
    mlp = NeuralHighlighter(args.depth, args.width, out_dim=args.n_classes, positional_encoding=args.positional_encoding,
                            sigma=args.sigma).to(device)
    optim = torch.optim.Adam(mlp.parameters(), args.learning_rate)

    # list of possible colors
    rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
    color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
    full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
    colors = torch.tensor(full_colors).to(device)


    # --- Prompt ---
    # pre-process multi_word_inputs
    args.object[0] = ' '.join(args.object[0].split('_'))
    for i in range(len(args.classes)):
        args.classes[i] = ' '.join(args.classes[i].split('_'))
    # encode prompt with CLIP
    prompt = "Point cloud depth map of gray {} with highlighted {}".format(args.object[0], args.classes[0])
    with torch.no_grad():
        prompt_token = clip.tokenize([prompt]).to(device)
        encoded_text = clip_model.encode_text(prompt_token)
        encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

    vertices = copy.deepcopy(verts)
    losses = []

    # Optimization loop
    for i in tqdm(range(args.n_iter)):
        optim.zero_grad()

        # predict highlight probabilities
        pred_class = mlp(vertices)

        # color and render mesh
        verts_curr = vertices
        colored_point_cloud = color_point_cloud(pred_class, verts_curr, colors)
        rendered_images = generate_static_views(colored_point_cloud, res)

        # Calculate CLIP Loss
        loss = clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model)
        loss.backward(retain_graph=True)

        optim.step()

        # update variables + record loss
        with torch.no_grad():
            losses.append(loss.item())

        # report results
        if i % 100 == 0:
            print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
            save_renders(dir, i, rendered_images)
            with open(os.path.join(dir, "training_info.txt"), "a") as f:
                f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")

    # re-initialize background color
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    # save results
    #save_final_results(args, dir, vertices, mlp, vertices, colors, render, background)

    # Save prompts
    with open(os.path.join(dir, prompt), "w") as f:
        f.write('')



# ================== HELPER FUNCTIONS =============================
def save_final_results(args, dir, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=args.n_views,
                                                                        show=args.show,
                                                                        center_azim=args.frontview_center[0],
                                                                        center_elev=args.frontview_center[1],
                                                                        std=args.frontview_std,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        objbase, extension = os.path.splitext(os.path.basename(args.obj_path))
        mesh.export(os.path.join(dir, f"{objbase}_{args.classes[0]}.ply"), extension="ply", color=final_color)
        save_renders(dir, 0, rendered_images, name='final_render.jpg')


def clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model):
    if args.n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if args.clipavg == "view":
            if encoded_text.shape[0] > 1:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)
        else:
            loss = torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    elif args.n_augs > 0:
        loss = 0.0
        for _ in range(args.n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if args.clipavg == "view":
                if encoded_text.shape[0] > 1:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
                else:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
            else:
                loss -= torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    return loss

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))



# OPTIMIZE MESH

In [8]:
import argparse
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision

from itertools import permutations, product
from neural_highlighter import NeuralHighlighter
from Normalization import MeshNormalizer,PointCloudNormalizer
from mesh import Mesh
from pathlib import Path
from render import Renderer
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device, color_mesh


def optimize(args):
    # Constrain most sources of randomness
    # (some torch backwards functions within CLIP are non-determinstic)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    # Load CLIP model
    clip_model, preprocess = clip.load(args.clipmodel, device, jit=args.jit)

    # Adjust output resolution depending on model type
    res = 224
    if args.clipmodel == "ViT-L/14@336px":
        res = 336
    if args.clipmodel == "RN50x4":
        res = 288
    if args.clipmodel == "RN50x16":
        res = 384
    if args.clipmodel == "RN50x64":
        res = 448

    Path(os.path.join(args.output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

    objbase, extension = os.path.splitext(os.path.basename(args.obj_path))

    render = Renderer(dim=(args.render_res, args.render_res))
    mesh = Mesh(args.obj_path)
    MeshNormalizer(mesh)()

    # Initialize variables
    background = None
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    n_augs = args.n_augs
    dir = args.output_dir

    # Record command line arguments
    with open(os.path.join(dir, 'commandline_args.txt'), 'w') as f:
        json.dump(args.__dict__, f, indent=2)

    # CLIP and Augmentation Transforms
    clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
    ])
    augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
    ])

    # MLP Settings
    mlp = NeuralHighlighter(args.depth, args.width, out_dim=args.n_classes, positional_encoding=args.positional_encoding,
                            sigma=args.sigma).to(device)
    optim = torch.optim.Adam(mlp.parameters(), args.learning_rate)

    # list of possible colors
    rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
    color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
    full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
    colors = torch.tensor(full_colors).to(device)


    # --- Prompt ---
    # pre-process multi_word_inputs
    args.object[0] = ' '.join(args.object[0].split('_'))
    for i in range(len(args.classes)):
        args.classes[i] = ' '.join(args.classes[i].split('_'))
    # encode prompt with CLIP
    prompt = "A 3D render of a gray {} with highlighted {}".format(args.object[0], args.classes[0])
    with torch.no_grad():
        prompt_token = clip.tokenize([prompt]).to(device)
        encoded_text = clip_model.encode_text(prompt_token)
        encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

    vertices = copy.deepcopy(mesh.vertices)
    losses = []

    # Optimization loop
    for i in tqdm(range(args.n_iter)):
        optim.zero_grad()

        # predict highlight probabilities
        pred_class = mlp(vertices)

        # color and render mesh
        sampled_mesh = mesh
        color_mesh(pred_class, sampled_mesh, colors)
        rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=args.n_views,
                                                                show=args.show,
                                                                center_azim=args.frontview_center[0],
                                                                center_elev=args.frontview_center[1],
                                                                std=args.frontview_std,
                                                                return_views=True,
                                                                lighting=True,
                                                                background=background)


        # Calculate CLIP Loss
        loss = clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model)
        loss.backward(retain_graph=True)

        optim.step()

        # update variables + record loss
        with torch.no_grad():
            losses.append(loss.item())

        # report results
        if i % 100 == 0:
            print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
            save_renders(dir, i, rendered_images)
            with open(os.path.join(dir, "training_info.txt"), "a") as f:
                f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")

    # re-initialize background color
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    # save results
    save_final_results(args, dir, mesh, mlp, vertices, colors, render, background)

    # Save prompts
    with open(os.path.join(dir, prompt), "w") as f:
        f.write('')



# ================== HELPER FUNCTIONS =============================
def save_final_results(args, dir, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=args.n_views,
                                                                        show=args.show,
                                                                        center_azim=args.frontview_center[0],
                                                                        center_elev=args.frontview_center[1],
                                                                        std=args.frontview_std,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        objbase, extension = os.path.splitext(os.path.basename(args.obj_path))
        mesh.export(os.path.join(dir, f"{objbase}_{args.classes[0]}.ply"), extension="ply", color=final_color)
        save_renders(dir, 0, rendered_images, name='final_render.jpg')


def clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model):
    if args.n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if args.clipavg == "view":
            if encoded_text.shape[0] > 1:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)
        else:
            loss = torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    elif args.n_augs > 0:
        loss = 0.0
        for _ in range(args.n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if args.clipavg == "view":
                if encoded_text.shape[0] > 1:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
                else:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
            else:
                loss -= torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    return loss

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))


# OPTIMIZE AFFORDANCE

In [9]:
import argparse
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
import sys
import os
import argparse
sys.path.append('/content/AML_Project_2024/3DHighlighter-main')
from itertools import permutations, product
from neural_highlighter import NeuralHighlighter
from Normalization import MeshNormalizer,PointCloudNormalizer
from mesh import Mesh
from pathlib import Path
from render import Renderer
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device, color_mesh
import open3d as o3d



def optimize_affordance(args, verts):
    # Constrain most sources of randomness
    # (some torch backwards functions within CLIP are non-determinstic)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    # Load CLIP model
    clip_model, preprocess = clip.load(args.clipmodel, device, jit=args.jit)

    # Adjust output resolution depending on model type
    res = 224
    if args.clipmodel == "ViT-L/14@336px":
        res = 336
    if args.clipmodel == "RN50x4":
        res = 288
    if args.clipmodel == "RN50x16":
        res = 384
    if args.clipmodel == "RN50x64":
        res = 448

    Path(os.path.join(args.output_dir, 'renders')).mkdir(parents=True, exist_ok=True)


    verts = verts.to(device)

    print("Verts shape")
    print(verts.shape)


    # Manually set RGB color (assuming you want to set all points to red)
    manual_color = torch.tensor([0.0, 1.0, 0.0], dtype=torch.float32, device=device)

    # Repeat the manual color for each vertex
    rgb_colors = manual_color.repeat(verts.shape[0], 1)

    shift = torch.mean(verts, dim=0)
    scale = torch.max(torch.norm(verts-shift, p=2, dim=1))
    verts=(verts-shift)/scale

    point_cloud = Pointclouds(points=[verts], features=[rgb_colors])
    #PointCloudNormalizer(point_cloud)()




    # Initialize variables
    background = None
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    n_augs = args.n_augs
    dir = args.output_dir

    # Record command line arguments
    with open(os.path.join(dir, 'commandline_args.txt'), 'w') as f:
        json.dump(args.__dict__, f, indent=2)

    # CLIP and Augmentation Transforms
    clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
    ])
    augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
    ])

    # MLP Settings
    mlp = NeuralHighlighter(args.depth, args.width, out_dim=args.n_classes, positional_encoding=args.positional_encoding,
                            sigma=args.sigma).to(device)
    optim = torch.optim.Adam(mlp.parameters(), args.learning_rate)

    # list of possible colors
    rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
    color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
    full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
    colors = torch.tensor(full_colors).to(device)


    # --- Prompt ---
    # pre-process multi_word_inputs
    args.object[0] = ' '.join(args.object[0].split('_'))
    for i in range(len(args.classes)):
        args.classes[i] = ' '.join(args.classes[i].split('_'))
    # encode prompt with CLIP
    #prompt = "If you were to open the Door, from which point on the door would you open it?"
    #prompt = "A point cloud representing a gray {} with highlighted  handle".format(args.object[0], args.classes[0])
    prompt = "A point cloud representing a gray {} with highlighted  handle".format(args.object[0])

    with torch.no_grad():
        prompt_token = clip.tokenize([prompt]).to(device)
        encoded_text = clip_model.encode_text(prompt_token)
        encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

    vertices = copy.deepcopy(verts)
    losses = []

    # Optimization loop
    for i in tqdm(range(args.n_iter)):
        optim.zero_grad()

        # predict highlight probabilities
        pred_class = mlp(vertices)

        # color and render mesh
        verts_curr = vertices
        colored_point_cloud = color_point_cloud(pred_class, verts_curr, colors)
        rendered_images =  generate_random_view(colored_point_cloud, res, args.n_views)

        # Calculate CLIP Loss
        loss = clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model)
        loss.backward(retain_graph=True)

        optim.step()

        # update variables + record loss
        with torch.no_grad():
            losses.append(loss.item())

        # report results
        if i % 100 == 0:
            print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
            save_renders(dir, i, rendered_images)
            with open(os.path.join(dir, "training_info.txt"), "a") as f:
                f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")

    # re-initialize background color
    if args.background is not None:
        assert len(args.background) == 3
        background = torch.tensor(args.background).to(device)
    # save results
    #save_final_results(args, dir, vertices, mlp, vertices, colors, render, background)

    # Save prompts
    with open(os.path.join(dir, prompt), "w") as f:
        f.write('')



# ================== HELPER FUNCTIONS =============================
def save_final_results(args, dir, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=args.n_views,
                                                                        show=args.show,
                                                                        center_azim=args.frontview_center[0],
                                                                        center_elev=args.frontview_center[1],
                                                                        std=args.frontview_std,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        objbase, extension = os.path.splitext(os.path.basename(args.obj_path))
        mesh.export(os.path.join(dir, f"{objbase}_{args.classes[0]}.ply"), extension="ply", color=final_color)
        save_renders(dir, 0, rendered_images, name='final_render.jpg')


def clip_loss(args, rendered_images, encoded_text, clip_transform, augment_transform, clip_model):
    if args.n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if args.clipavg == "view":
            if encoded_text.shape[0] > 1:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)
        else:
            loss = torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    elif args.n_augs > 0:
        loss = 0.0
        for _ in range(args.n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if args.clipavg == "view":
                if encoded_text.shape[0] > 1:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
                else:
                    loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
            else:
                loss -= torch.mean(torch.cosine_similarity(encoded_renders, encoded_text))
    return loss

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))



# MAIN

In [10]:
import sys
import os
import argparse
#sys.path.append('/content/AML_Project_2024/3DHighlighter-main/')
#from main import optimize # Import the function from the module

class ArgsNamespace:
    def __init__(self, args_dict):
        self.__dict__.update(args_dict)

# Create an instance of ArgsNamespace



# specify the directory you want to iterate over
directory = './AML_Project_2024/point_cloud_data/'  # current directory
#directory= './AML_Project_2024/dataset/'

# iterate over files in the directory

for filename in os.listdir(directory):
    if(filename.split('.')[0]!='bull'):
      continue

    object_name = filename.split('.')[0]
    print(object_name)


    args = ArgsNamespace({
        'seed': 0,
        'obj_path': directory + filename,
        'output_dir': '/content/results3_' + object_name,
        'object': [object_name],
        'classes': ['head'],
        'background': [1., 1., 1.],
        'n_views': 5,
        'frontview_std': 4,
        'frontview_center': [0., 0.],
        'show': False,
        'n_augs': 1,
        'clipavg': 'view',
        'render_res': 224,
        'clipmodel': 'ViT-L/14',
        'jit': False,
        'depth': 4,
        'width': 256,
        'n_classes': 2,
        'positional_encoding': False,
        'sigma': 5.0,
        'learning_rate': 0.0001,
        'n_iter': 1500
    })


    optimize_point_cloud(args)

"""

from google.colab import files

!zip -r /content/results_television.zip /content/results_television
files.download('/content/results_television.zip')

"""

FileNotFoundError: [Errno 2] No such file or directory: './AML_Project_2024/point_cloud_data/'

# MAIN AFFORDANCE

In [13]:
import sys
import os
import argparse
import pickle as pkl
import torch
import numpy as np

#sys.path.append('/content/AML_Project_2024/3DHighlighter-main/')
#from main import optimize # Import the function from the module

class ArgsNamespace:
    def __init__(self, args_dict):
        self.__dict__.update(args_dict)

# Create an instance of ArgsNamespace



# specify the directory you want to iterate over
#directory = './AML_Project_2024/point_cloud_dataset/'  # current directory
#directory= './AML_Project_2024/dataset/'
directory = './AML_Project_2024/full_shape_val_data/Door/'
# iterate over files in the directory


for filename in os.listdir(directory):

    with open(directory + filename, 'rb') as f: ## da definire...
      # Load the object from the file
      temp_data = pkl.load(f)
      # print((temp_data[0]))
      verts = temp_data[10]["full_shape"]["coordinate"]
      verts = torch.Tensor(np.asarray(verts))


    object_name = temp_data[10]["semantic class"]
    print(object_name)


    args = ArgsNamespace({
        'seed': 0,
        'obj_path': directory + filename,
        'output_dir': '/content/resu3l2ts2110_' + object_name,
        'object': [object_name],
        'classes': ["open"],
        'background': [1., 1., 1.],
        'n_views': 5,
        'frontview_std': 4,
        'frontview_center': [0., 0.],
        'show': False,
        'n_augs': 0,
        'clipavg': 'view',
        'render_res': 224,
        'clipmodel': 'ViT-L/14',
        'jit': False,
        'depth': 4,
        'width': 256,
        'n_classes': 2,
        'positional_encoding': False,
        'sigma': 5.0,
        'learning_rate': 0.0001,
        'n_iter': 2500
    })


    optimize_affordance(args, verts)
    break

"""

from google.colab import files

!zip -r /content/results_television.zip /content/results_television
files.download('/content/results_television.zip')

"""

Door
Verts shape
torch.Size([2048, 3])
ModuleList(
  (0): Linear(in_features=3, out_features=256, bias=True)
  (1): ReLU()
  (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (9): Linear(in_features=256, out_features=256, bias=True)
  (10): ReLU()
  (11): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (12): Linear(in_features=256, out_features=256, bias=True)
  (13): ReLU()
  (14): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (15): Linear(in_features=256, out_features=2, bias=True)
  (16): Softmax(dim=1)
)


  0%|          | 1/2500 [00:00<15:59,  2.61it/s]

Last 100 CLIP score: 0.195068359375


  4%|▍         | 101/2500 [00:30<12:20,  3.24it/s]

Last 100 CLIP score: 0.204022216796875


...

Last 100 CLIP score: 0.20357421875


 96%|█████████▌| 2401/2500 [11:46<00:30,  3.28it/s]

Last 100 CLIP score: 0.201729736328125


100%|██████████| 2500/2500 [12:16<00:00,  3.40it/s]


"\n\nfrom google.colab import files\n\n!zip -r /content/results_television.zip /content/results_television\nfiles.download('/content/results_television.zip')\n\n"